In [14]:
!pip install pdf2image transformers torch torchvision
!pip install reportlab PyPDF2
!sudo apt install poppler-utils  # Required for pdf2image (Linux)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.1 MB/s eta 0:00:00a 0:00:01
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118
The following NEW packages will be installed:
  poppler-utils
The following packages will be upgraded:
  libpoppler-dev libpoppler-private-dev libpoppler118
3 upgraded, 1 newly installed, 0 to remove and 126 not upgraded.
Need to get 1,462 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-private-dev amd64 22.02.0-2ubuntu0.6 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-dev amd64 22.02.0-2ubuntu0.6 [5,184 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.6 [1,071 kB]
Get:4 http://archive.ubuntu.com/ubunt

In [2]:
from pdf2image import convert_from_path

# Convert PDF to image
images = convert_from_path('/kaggle/input/sample-contract-form/Sample_form_fillable.pdf')
images[0].save('page1.png', 'PNG')  # Save first page


In [17]:
!rm -rf /kaggle/working/overlay_filled.pdf
!rm -rf /kaggle/working/final_filled_form_corrected.pdf
!rm -rf /kaggle/working/final_filled_form_precise.pdf

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from PIL import Image
import torch

model_id = "Qwen/Qwen2-VL-2B"

# Load the 2B vision-language model and processor
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch.float16, device_map="auto"
)

In [ ]:
save_path="qwen2vl_2B"

In [ ]:
# removing redundant files
!rm -rf {save_path}



In [ ]:
!mkdir {save_path}

In [ ]:
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
processor.save_pretrained(save_path)

In [7]:
import re

def extract_json_array(text: str):
    match = re.search(r"\[\s*{.*?}\s*\]", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass
    return []


In [22]:
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from PIL import Image
import torch
import json

# Load model and processor
model_id = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_id)

# Load image
image = Image.open("page1.png").convert("RGB").resize((1024, 1024))

input_json = {
    "Name": "Sai Srinivas",
    "Age": 26,
    "Married": "No",
    "Gender": "Male"
}

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": (
                    "You are analyzing a scanned form with fillable fields like checkboxes, radio buttons, and text inputs. "
                    "Each field appears in a structured table where rows represent individuals and columns represent attributes like Name, Age, Gender, Marital Status, and Children.\n\n"

                    "Your task is to:\n"
                    "1. Detect **all fillable fields visually** from the form image.\n"
                    "2. For each field, extract its **visible label** and the **exact bounding box** in pixel coordinates.\n"
                    "3. Match fields with the provided input JSON and determine if they should be filled.\n"
                    "4. Return only a JSON array where each item includes:\n"
                    "   - 'field_type': (e.g., 'checkbox', 'radio', 'text')\n"
                    "   - 'label': visible text label near the field\n"
                    "   - 'bbox': [x1, y1, x2, y2] in pixel coordinates — top-left origin\n"
                    "   - 'meta': [column header, row name]\n"
                    "   - 'fill': true or false\n\n"

                    "📐 Bounding Box Instructions:\n"
                    "- All `bbox` values must be visually extracted from the image.\n"
                    "- Use absolute pixel positions (e.g., [120, 412, 140, 428])\n"
                    "- Do NOT use `...` or `[...]`. Always return real numeric values.\n\n"

                    "🧠 Reasoning Rules:\n"
                    "- If Age ≥ 18 → fill '≥ 18'; if < 18 → fill '< 18'\n"
                    "- If Married = 'No' → fill 'Single'; if 'Yes' → fill 'Married'\n"
                    "- If Children ≥ 1 → fill 'Children'; if missing or 0 → false\n"
                    "- If Married = 'No' and Children is missing, assume no children\n"
                    "- Only set `fill: true` if supported by logic or input\n\n"

                    "🔁 Output Requirement:\n"
                    "- You must return **all visually detected fillable fields**.\n"
                    "- Even if a field is unrelated or logically false — include it with `fill: false`\n"
                    "- Do not skip or filter any field — always return all with correct fill\n"
                    "- Return only a clean JSON array — no explanations, markdown, or headers\n\n"

                    "✅ Few-Shot Examples:\n\n"

                    "🔹 Example 1 — Adult single, no children\n"
                    "Input:\n"
                    "{ \"Name\": \"Sai Srinivas\", \"Age\": 26, \"Married\": \"No\", \"Gender\": \"Male\", \"Children\": 0 }\n"
                    "Output:\n"
                    "[\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"≥ 18\", \"bbox\": [123, 412, 135, 428], \"meta\": [\"Age\", \"Sai Srinivas\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"< 18\", \"bbox\": [140, 412, 155, 428], \"meta\": [\"Age\", \"Sai Srinivas\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Single\", \"bbox\": [175, 412, 190, 428], \"meta\": [\"Marital Status\", \"Sai Srinivas\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Married\", \"bbox\": [200, 412, 215, 428], \"meta\": [\"Marital Status\", \"Sai Srinivas\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Children\", \"bbox\": [320, 412, 335, 428], \"meta\": [\"Children\", \"Sai Srinivas\"], \"fill\": false }\n"
                    "]\n\n"

                    "🔹 Example 2 — Married with 2 children\n"
                    "Input:\n"
                    "{ \"Name\": \"Anita\", \"Age\": 35, \"Married\": \"Yes\", \"Children\": 2 }\n"
                    "Output:\n"
                    "[\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"≥ 18\", \"bbox\": [123, 370, 135, 388], \"meta\": [\"Age\", \"Anita\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"< 18\", \"bbox\": [140, 370, 155, 388], \"meta\": [\"Age\", \"Anita\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Married\", \"bbox\": [200, 370, 215, 388], \"meta\": [\"Marital Status\", \"Anita\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Single\", \"bbox\": [175, 370, 190, 388], \"meta\": [\"Marital Status\", \"Anita\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Children\", \"bbox\": [320, 370, 335, 388], \"meta\": [\"Children\", \"Anita\"], \"fill\": true }\n"
                    "]\n\n"

                    "🔹 Example 3 — Single, but Children key not provided\n"
                    "Input:\n"
                    "{ \"Name\": \"Vikram\", \"Age\": 22, \"Married\": \"No\" }\n"
                    "Output:\n"
                    "[\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"≥ 18\", \"bbox\": [123, 330, 135, 348], \"meta\": [\"Age\", \"Vikram\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"< 18\", \"bbox\": [140, 330, 155, 348], \"meta\": [\"Age\", \"Vikram\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Single\", \"bbox\": [175, 330, 190, 348], \"meta\": [\"Marital Status\", \"Vikram\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Married\", \"bbox\": [200, 330, 215, 348], \"meta\": [\"Marital Status\", \"Vikram\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Children\", \"bbox\": [320, 330, 335, 348], \"meta\": [\"Children\", \"Vikram\"], \"fill\": false }\n"
                    "]\n\n"

                    "🔹 Example 4 — Married, but Children = 0\n"
                    "Input:\n"
                    "{ \"Name\": \"Kiran\", \"Age\": 40, \"Married\": \"Yes\", \"Children\": 0 }\n"
                    "Output:\n"
                    "[\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"≥ 18\", \"bbox\": [123, 290, 135, 308], \"meta\": [\"Age\", \"Kiran\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"< 18\", \"bbox\": [140, 290, 155, 308], \"meta\": [\"Age\", \"Kiran\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Married\", \"bbox\": [200, 290, 215, 308], \"meta\": [\"Marital Status\", \"Kiran\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Single\", \"bbox\": [175, 290, 190, 308], \"meta\": [\"Marital Status\", \"Kiran\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Children\", \"bbox\": [320, 290, 335, 308], \"meta\": [\"Children\", \"Kiran\"], \"fill\": false }\n"
                    "]\n\n"

                    "🔹 Example 5 — Teenager, Married key missing\n"
                    "Input:\n"
                    "{ \"Name\": \"Rita\", \"Age\": 15, \"Gender\": \"Female\" }\n"
                    "Output:\n"
                    "[\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"< 18\", \"bbox\": [123, 250, 135, 268], \"meta\": [\"Age\", \"Rita\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"≥ 18\", \"bbox\": [140, 250, 155, 268], \"meta\": [\"Age\", \"Rita\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Single\", \"bbox\": [175, 250, 190, 268], \"meta\": [\"Marital Status\", \"Rita\"], \"fill\": true },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Married\", \"bbox\": [200, 250, 215, 268], \"meta\": [\"Marital Status\", \"Rita\"], \"fill\": false },\n"
                    "  { \"field_type\": \"checkbox\", \"label\": \"Children\", \"bbox\": [320, 250, 335, 268], \"meta\": [\"Children\", \"Rita\"], \"fill\": false }\n"
                    "]\n\n"

                    f"Now apply this logic to the following input JSON:\n{json.dumps(input_json, indent=2)}\n\n"
                    "Return only the JSON array. Do not include any explanations, markdown, or formatting."
                )
            }
        ]
    }
]





# Convert messages to text format using the processor
text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

# Encode both prompt and image
inputs = processor(
    text=[text_prompt],
    images=[image],
    return_tensors="pt",
    padding=True
).to("cuda")


# Generate
generate_ids = model.generate(**inputs, max_new_tokens=1024)
raw_output = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]
print("🧠 Raw model output:\n", raw_output)







Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🧠 Raw model output:
 system
You are a helpful assistant.
user
You are analyzing a scanned form with fillable fields like checkboxes, radio buttons, and text inputs. Each field appears in a structured table where rows represent individuals and columns represent attributes like Name, Age, Gender, Marital Status, and Children.

Your task is to:
1. Detect **all fillable fields visually** from the form image.
2. For each field, extract its **visible label** and the **exact bounding box** in pixel coordinates.
3. Match fields with the provided input JSON and determine if they should be filled.
4. Return only a JSON array where each item includes:
   - 'field_type': (e.g., 'checkbox', 'radio', 'text')
   - 'label': visible text label near the field
   - 'bbox': [x1, y1, x2, y2] in pixel coordinates — top-left origin
   - 'meta': [column header, row name]
   - 'fill': true or false

📐 Bounding Box Instructions:
- All `bbox` values must be visually extracted from the image.
- Use absolute pixel

In [3]:
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from PIL import Image
import torch
import json

# Load model and processor
model_id = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_id)

# Load image
image = Image.open("page1.png").convert("RGB").resize((1024, 1024))

input_json = {
    "Name": "Sai Srinivas",
    "Age": 26,
    "Married": "No",
    "Gender": "Male"
}

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": (
                    "You are analyzing a scanned form with fillable fields such as checkboxes and text inputs. "
                    "The form is structured as a table where rows represent individuals and columns represent attributes.\n\n"

                    "📋 Fillable Field Reference:\n"
                    "There are exactly 11 fillable fields:\n"
                    "- Gender: Male, Female, Others (checkbox)\n"
                    "- Age group: Below 18, ≥ 18 (checkbox)\n"
                    "- Marital Status: Married, Single (checkbox)\n"
                    "- Children: 1, 2, ≥ 3 (checkbox)\n"
                    "- Name: Sp.Name (text input)\n\n"

                    "🧠 Your task:\n"
                    "1. Detect **all** of the above fields in the image.\n"
                    "2. For each, extract:\n"
                    "   - 'field_type': e.g. 'checkbox', 'text'\n"
                    "   - 'label': as shown in the form (e.g., 'Male', '≥ 18')\n"
                    "   - 'bbox': bounding box in format [x1, y1, x2, y2] from the image\n"
                    "   - 'meta': [column header, row name]\n"
                    "   - 'fill': True if the input JSON supports or implies filling it, else False\n\n"

                    "📐 Bbox Format:\n"
                    "- Use image pixel coordinates with top-left as (0, 0)\n"
                    "- Do not return `[...]` or `...`. All values must be numeric.\n\n"

                    "🔁 Output Rule:\n"
                    "- Always return all 11 fields — even if `fill: false`\n"
                    "- Do not invent or omit labels\n"
                    "- Return only a JSON array, no markdown or explanation\n\n"

                    "✅ Few-Shot Example — Single adult, no children key\n"
                    "Input:\n"
                    "{ \"Name\": \"Sai Srinivas\", \"Age\": 26, \"Married\": \"No\", \"Gender\": \"Male\" }\n"
                    "Output:\n"
                    "[\n"
                    "  {\"field_type\": \"text\", \"label\": \"Sp.Name\", \"bbox\": [100, 200, 300, 220], \"meta\": [\"Name\", \"Sai Srinivas\"], \"fill\": True},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"Male\", \"bbox\": [120, 250, 135, 265], \"meta\": [\"Gender\", \"Sai Srinivas\"], \"fill\": True},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"Female\", \"bbox\": [140, 250, 155, 265], \"meta\": [\"Gender\", \"Sai Srinivas\"], \"fill\": False},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"Others\", \"bbox\": [160, 250, 175, 265], \"meta\": [\"Gender\", \"Sai Srinivas\"], \"fill\": False},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"Below 18\", \"bbox\": [180, 250, 195, 265], \"meta\": [\"Age group\", \"Sai Srinivas\"], \"fill\": False},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"≥ 18\", \"bbox\": [200, 250, 215, 265], \"meta\": [\"Age group\", \"Sai Srinivas\"], \"fill\": True},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"Single\", \"bbox\": [220, 250, 235, 265], \"meta\": [\"Marital Status\", \"Sai Srinivas\"], \"fill\": True},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"Married\", \"bbox\": [240, 250, 255, 265], \"meta\": [\"Marital Status\", \"Sai Srinivas\"], \"fill\": False},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"1\", \"bbox\": [260, 250, 275, 265], \"meta\": [\"Children\", \"Sai Srinivas\"], \"fill\": False},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"2\", \"bbox\": [280, 250, 295, 265], \"meta\": [\"Children\", \"Sai Srinivas\"], \"fill\": False},\n"
                    "  {\"field_type\": \"checkbox\", \"label\": \"≥ 3\", \"bbox\": [300, 250, 315, 265], \"meta\": [\"Children\", \"Sai Srinivas\"], \"fill\": False}\n"
                    "]\n\n"

                    f"Now apply this to:\n{json.dumps(input_json, indent=2)}\n\n"
                    "Return only the JSON array."
                )
            }
        ]
    }
]






# Convert messages to text format using the processor
text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

# Encode both prompt and image
inputs = processor(
    text=[text_prompt],
    images=[image],
    return_tensors="pt",
    padding=True
).to("cuda")


# Generate
generate_ids = model.generate(**inputs, max_new_tokens=1024)
raw_output = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]
print("🧠 Raw model output:\n", raw_output)







config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

🧠 Raw model output:
 system
You are a helpful assistant.
user
You are analyzing a scanned form with fillable fields such as checkboxes and text inputs. The form is structured as a table where rows represent individuals and columns represent attributes.

📋 Fillable Field Reference:
There are exactly 11 fillable fields:
- Gender: Male, Female, Others (checkbox)
- Age group: Below 18, ≥ 18 (checkbox)
- Marital Status: Married, Single (checkbox)
- Children: 1, 2, ≥ 3 (checkbox)
- Name: Sp.Name (text input)

🧠 Your task:
1. Detect **all** of the above fields in the image.
2. For each, extract:
   - 'field_type': e.g. 'checkbox', 'text'
   - 'label': as shown in the form (e.g., 'Male', '≥ 18')
   - 'bbox': bounding box in format [x1, y1, x2, y2] from the image
   - 'meta': [column header, row name]
   - 'fill': true if the input JSON supports or implies filling it, else false

📐 Bbox Format:
- Use image pixel coordinates with top-left as (0, 0)
- Do not return `[...]` or `...`. All values mu

In [5]:
import re
import json

def extract_json_array_from_qwen_output(raw_output):
    # Look for the final block after the last "assistant" marker
    assistant_split = raw_output.strip().split("assistant")[-1]

    # Find the first valid JSON array after that
    matches = re.findall(r"\[\s*{.*?}\s*]", assistant_split, re.DOTALL)
    for match in matches:
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue
    return []


In [6]:
response = extract_json_array_from_qwen_output(raw_output)
if not response:
    print("⚠️ Still no valid JSON extracted.")
else:
    print("✅ Extracted fields:\n", json.dumps(response, indent=2))

✅ Extracted fields:
 [
  {
    "field_type": "text",
    "label": "Sp.Name",
    "bbox": [
      100,
      200,
      300,
      220
    ],
    "meta": [
      "Name",
      "Sai Srinivas"
    ],
    "fill": true
  },
  {
    "field_type": "checkbox",
    "label": "Male",
    "bbox": [
      120,
      250,
      135,
      265
    ],
    "meta": [
      "Gender",
      "Sai Srinivas"
    ],
    "fill": true
  },
  {
    "field_type": "checkbox",
    "label": "Female",
    "bbox": [
      140,
      250,
      155,
      265
    ],
    "meta": [
      "Gender",
      "Sai Srinivas"
    ],
    "fill": false
  },
  {
    "field_type": "checkbox",
    "label": "Others",
    "bbox": [
      160,
      250,
      175,
      265
    ],
    "meta": [
      "Gender",
      "Sai Srinivas"
    ],
    "fill": false
  },
  {
    "field_type": "checkbox",
    "label": "Below 18",
    "bbox": [
      180,
      250,
      195,
      265
    ],
    "meta": [
      "Age group",
      "Sai Srinivas"
 

In [7]:
!pip install pdfrw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 5.6 MB/s eta 0:00:00


In [18]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from pdfrw import PdfReader, PdfWriter, PageMerge
import io

# Use this map to override bboxes
pdf_field_positions = {
    ("Gender", "Male"):        [172, 724, 184, 736],
    ("Gender", "Female"):      [220, 724, 232, 736],
    ("Gender", "Others"):      [270, 724, 282, 736],
    ("Age group", "Below 18"): [172, 688, 190, 700],
    ("Age group", "≥ 18"):     [220, 688, 238, 700],
    ("Marital Status", "Married"): [172, 652, 194, 664],
    ("Marital Status", "Single"):  [220, 652, 242, 664],
    ("Children", "1"):         [172, 616, 184, 628],
    ("Children", "2"):         [220, 616, 232, 628],
    ("Children", "≥ 3"):       [270, 616, 282, 628],
    ("Name", "Sp.Name"):       [172, 580, 272, 600],
}
pdf_field_positions.update({
    ("Age group", "≥ 18"):     [315, 688, 327, 700],     # was too left
    ("Marital Status", "Single"):  [315, 652, 327, 664],  # shifted right
})

# --- Create overlay with corrected positions ---
packet = io.BytesIO()
c = canvas.Canvas(packet, pagesize=letter)

for field in response:
    if not field.get("fill"):
        continue

    label = field["label"]
    meta = field.get("meta", [])
    if len(meta) != 2:
        continue

    col, row = meta
    key = (col, label)

    if key not in pdf_field_positions:
        continue  # Skip unknown

    x1, y1, x2, y2 = pdf_field_positions[key]
    w, h = x2 - x1, y2 - y1
    cx, cy = x1 + w / 2, y1 + h / 2  # center

    if field["field_type"] == "checkbox":
        c.setFont("Helvetica-Bold", 12)
        c.drawCentredString(cx, cy - 4, "✔")

    elif field["field_type"] == "text":
        c.setFont("Helvetica", 10)
        c.drawString(x1 + 2, y1 + 2, label)

c.save()
packet.seek(0)

# --- Merge with original PDF ---
original = PdfReader("/kaggle/input/sample-contract-form/Sample_form_fillable.pdf")
overlay = PdfReader(packet)
output = PdfWriter()

PageMerge(original.pages[0]).add(overlay.pages[0]).render()
output.addpage(original.pages[0])
output.write("/kaggle/working/final_filled_form_precise.pdf")

print("✅ Final form saved with corrected tick positions.")


✅ Final form saved with corrected tick positions.


In [2]:
!pip install PyPDF2 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.1 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfReader

# Path to your form PDF
pdf_path = "/kaggle/input/sample-contract-form/Sample_form_fillable.pdf"

# Load the PDF
reader = PdfReader(pdf_path)
page = reader.pages[0]

# Extract annotations (form fields)
fields_info = []
for annot in page.get("/Annots", []):
    obj = annot.get_object()
    rect = obj.get("/Rect")         # Bounding box
    name = obj.get("/T")            # Field name
    field_type = obj.get("/FT")     # Field type (Tx, Btn, etc.)
    subtype = obj.get("/Subtype")   # Widget type (usually /Widget)

    # Collect only form widgets
    if subtype == "/Widget":
        fields_info.append({
            "name": name if name else "(unnamed)",
            "type": field_type if field_type else "(unknown)",
            "rect": rect
        })

# Display all extracted fields with positions
for i, field in enumerate(fields_info, 1):
    print(f"{i}. Name: {field['name']}, Type: {field['type']}, Rect: {field['rect']}")


1. Name: (unnamed), Type: (unknown), Rect: [395, 695, 444, 710]
2. Name: (unnamed), Type: (unknown), Rect: [171, 724, 178, 731]
3. Name: (unnamed), Type: (unknown), Rect: [265, 725, 272, 732]
4. Name: (unnamed), Type: (unknown), Rect: [358, 725, 365, 732]
5. Name: (unnamed), Type: (unknown), Rect: [451, 725, 458, 732]
6. Name: (unnamed), Type: (unknown), Rect: [172, 710, 179, 717]
7. Name: (unnamed), Type: (unknown), Rect: [264, 710, 271, 717]
8. Name: (unnamed), Type: (unknown), Rect: [358, 709, 365, 716]
9. Name: (unnamed), Type: (unknown), Rect: [451, 710, 458, 717]
10. Name: (unnamed), Type: (unknown), Rect: [172, 694, 179, 701]
11. Name: (unnamed), Type: (unknown), Rect: [450, 696, 457, 703]


In [26]:
import io
import json
import re
from pdfrw import PdfReader, PdfWriter, PageMerge
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

# --- Utility to extract clean JSON array from model output ---
'''def extract_json_array(text: str):
    match = re.search(r"\[\s*{.*?}\s*\]", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass
    return []

# --- Assume raw_output exists from model ---
response = extract_json_array(raw_output)'''

# --- Create overlay PDF in memory ---
packet = io.BytesIO()
c = canvas.Canvas(packet, pagesize=letter)

for field in response:
    if not field.get("fill"):
        continue

    try:
        x1, y1, x2, y2 = field["bbox"]
        w, h = x2 - x1, y2 - y1
        field_type = field.get("field_type", "").lower()
        label = field.get("label", "")

        if field_type in ["checkbox", "radio"]:
            c.setFont("Helvetica", min(h, 14))
            c.drawString(x1 + 1, y1 + 1, "✔")

        elif field_type == "text":
            c.setFont("Helvetica", min(h, 10))
            c.drawString(x1 + 1, y1 + h * 0.25, label)

    except Exception as e:
        print(f"⚠️ Skipping field due to error: {e}")

c.save()
packet.seek(0)

# --- Load original PDF and overlay ---
original_pdf_path = "/kaggle/input/sample-contract-form/Sample_form_fillable.pdf"
output_pdf_path = "/kaggle/working/final_filled_form.pdf"

pdf = PdfReader(original_pdf_path)
overlay = PdfReader(packet)

for page_num, page in enumerate(pdf.pages):
    merger = PageMerge(page)
    merger.add(overlay.pages[0]).render()

# --- Save the final flattened output ---
PdfWriter(output_pdf_path, trailer=pdf).write()
print(f"✅ Flattened filled PDF saved as: {output_pdf_path}")


✅ Flattened filled PDF saved as: /kaggle/working/final_filled_form.pdf


In [11]:
from PyPDF2 import PdfReader

reader = PdfReader("/kaggle/input/sample-contract-form/Sample_form_fillable.pdf")
acroform = reader.trailer["/Root"].get("/AcroForm", None)

pdf_fields = []

if acroform:
    form_obj = acroform.get_object()
    fields = form_obj.get("/Fields", [])
    print("✅ /AcroForm fields found:\n")

    for i, field in enumerate(fields, 1):
        field_obj = field.get_object()
        name = field_obj.get("/T")
        field_type = field_obj.get("/FT")

        # Get bounding box
        rect = field_obj.get("/Rect")
        if not rect and "/Kids" in field_obj:
            kids = field_obj["/Kids"]
            if kids:
                kid_obj = kids[0].get_object()
                rect = kid_obj.get("/Rect")

        # Only include if rect is valid
        if rect:
            rect_coords = [float(v) for v in rect]
            pdf_fields.append({
                "name": name if name else "(unnamed)",
                "type": field_type if field_type else "(unknown)",
                "rect": rect_coords
            })

# Output as list of dicts
import json
print(json.dumps(pdf_fields, indent=2))


✅ /AcroForm fields found:

[
  {
    "name": "below 18",
    "type": "/Btn",
    "rect": [
      265.0,
      725.0,
      272.0,
      732.0
    ]
  },
  {
    "name": "married",
    "type": "/Btn",
    "rect": [
      358.0,
      725.0,
      365.0,
      732.0
    ]
  },
  {
    "name": "1",
    "type": "/Btn",
    "rect": [
      451.0,
      725.0,
      458.0,
      732.0
    ]
  },
  {
    "name": "female",
    "type": "/Btn",
    "rect": [
      172.0,
      710.0,
      179.0,
      717.0
    ]
  },
  {
    "name": "single",
    "type": "/Btn",
    "rect": [
      358.0,
      709.0,
      365.0,
      716.0
    ]
  },
  {
    "name": "2",
    "type": "/Btn",
    "rect": [
      451.0,
      710.0,
      458.0,
      717.0
    ]
  },
  {
    "name": "others",
    "type": "/Btn",
    "rect": [
      172.0,
      694.0,
      179.0,
      701.0
    ]
  },
  {
    "name": "text_15tciq",
    "type": "/Tx",
    "rect": [
      394.0,
      695.0,
      443.0,
      710.0
    ]
  }

In [12]:
import json

# ---- Step 1: QWEN Output (model-generated) ----
response = [
  {
    "field_type": "text",
    "label": "Sp.Name",
    "bbox": [100, 200, 300, 220],
    "meta": ["Name", "Sai Srinivas"],
    "fill": True
  },
  {
    "field_type": "checkbox",
    "label": "Male",
    "bbox": [120, 250, 135, 265],
    "meta": ["Gender", "Sai Srinivas"],
    "fill": True
  },
  {
    "field_type": "checkbox",
    "label": "Female",
    "bbox": [140, 250, 155, 265],
    "meta": ["Gender", "Sai Srinivas"],
    "fill": False
  },
  {
    "field_type": "checkbox",
    "label": "Others",
    "bbox": [160, 250, 175, 265],
    "meta": ["Gender", "Sai Srinivas"],
    "fill": False
  },
  {
    "field_type": "checkbox",
    "label": "Below 18",
    "bbox": [180, 250, 195, 265],
    "meta": ["Age group", "Sai Srinivas"],
    "fill": False
  },
  {
    "field_type": "checkbox",
    "label": "≥ 18",
    "bbox": [200, 250, 215, 265],
    "meta": ["Age group", "Sai Srinivas"],
    "fill": True
  },
  {
    "field_type": "checkbox",
    "label": "Single",
    "bbox": [220, 250, 235, 265],
    "meta": ["Marital Status", "Sai Srinivas"],
    "fill": True
  },
  {
    "field_type": "checkbox",
    "label": "Married",
    "bbox": [240, 250, 255, 265],
    "meta": ["Marital Status", "Sai Srinivas"],
    "fill": False
  },
  {
    "field_type": "checkbox",
    "label": "1",
    "bbox": [260, 250, 275, 265],
    "meta": ["Children", "Sai Srinivas"],
    "fill": False
  },
  {
    "field_type": "checkbox",
    "label": "2",
    "bbox": [280, 250, 295, 265],
    "meta": ["Children", "Sai Srinivas"],
    "fill": False
  },
  {
    "field_type": "checkbox",
    "label": "≥ 3",
    "bbox": [300, 250, 315, 265],
    "meta": ["Children", "Sai Srinivas"],
    "fill": False
  }
]



# ---- Step 2: Actual Field Info from PDF (PyPDF2 extraction) ----
'''pdf_fields = [
    {"name": "male", "type": "/Btn", "rect": [171, 724, 178, 731]},
    {"name": "female", "type": "/Btn", "rect": [172, 710, 179, 717]},
    {"name": "others", "type": "/Btn", "rect": [172, 694, 179, 701]},
    {"name": "below 18", "type": "/Btn", "rect": [265, 725, 272, 732]},
    {"name": "18 and above", "type": "/Btn", "rect": [264, 710, 271, 717]},
    {"name": "single", "type": "/Btn", "rect": [358, 709, 365, 716]},
    {"name": "married", "type": "/Btn", "rect": [358, 725, 365, 732]},
    {"name": "1", "type": "/Btn", "rect": [451, 725, 458, 732]},
    {"name": "2", "type": "/Btn", "rect": [451, 710, 458, 717]},
    {"name": "3 or more", "type": "/Btn", "rect": [450, 696, 457, 703]},
    {"name": "text_15tciq", "type": "/Tx", "rect": [395, 695, 444, 710]},
]'''

# ---- Step 3: Normalize Keys for Matching ----
def normalize(label):
    return label.strip().lower().replace("≥", "18 and above").replace("<", "below").replace(">", "more").replace("=", "").replace("  ", " ").replace("≥ 3", "3 or more")

# ---- Step 4: Match and Replace bbox ----
updated_response = []
for field in response:
    label = normalize(field["label"])
    field_type = field["field_type"]

    # Search for best match
    matched = None
    for pdf_field in pdf_fields:
        if normalize(pdf_field["name"]) == label:
            matched = pdf_field
            break

    # Replace bbox if match found
    if matched:
        field["bbox"] = matched["rect"]
    else:
        print(f"⚠️ No match found for: {field['label']}")
    
    updated_response.append(field)

# ---- Step 5: Output Final JSON ----
print(json.dumps(updated_response, indent=2))


⚠️ No match found for: Sp.Name
[
  {
    "field_type": "text",
    "label": "Sp.Name",
    "bbox": [
      100,
      200,
      300,
      220
    ],
    "meta": [
      "Name",
      "Sai Srinivas"
    ],
    "fill": true
  },
  {
    "field_type": "checkbox",
    "label": "Male",
    "bbox": [
      171.0,
      724.0,
      178.0,
      731.0
    ],
    "meta": [
      "Gender",
      "Sai Srinivas"
    ],
    "fill": true
  },
  {
    "field_type": "checkbox",
    "label": "Female",
    "bbox": [
      172.0,
      710.0,
      179.0,
      717.0
    ],
    "meta": [
      "Gender",
      "Sai Srinivas"
    ],
    "fill": false
  },
  {
    "field_type": "checkbox",
    "label": "Others",
    "bbox": [
      172.0,
      694.0,
      179.0,
      701.0
    ],
    "meta": [
      "Gender",
      "Sai Srinivas"
    ],
    "fill": false
  },
  {
    "field_type": "checkbox",
    "label": "Below 18",
    "bbox": [
      265.0,
      725.0,
      272.0,
      732.0
    ],
    "meta": [

In [16]:
!pip install pdfrw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 3.0 MB/s eta 0:00:00


In [18]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from pdfrw import PdfReader, PdfWriter, PageMerge
import io

# === Your response data ===
'''response = [
    # Paste your full response dictionary here (as already shared above)
]'''

# === Step 1: Create Overlay with Ticks ===
packet = io.BytesIO()
c = canvas.Canvas(packet, pagesize=letter)

# Draw ticks for each field where fill is True
for field in updated_response:
    if field["fill"]:
        x1, y1, x2, y2 = field["bbox"]
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        c.setFont("Helvetica-Bold", 12)
        c.drawCentredString(center_x, center_y - 4, "✔")

c.save()
packet.seek(0)

# === Step 2: Merge Overlay with Original PDF ===
original_pdf_path = "/kaggle/input/sample-contract-form/Sample_form_fillable.pdf"  # update if needed
output_pdf_path = "/kaggle/working/final_filled_form.pdf"

base_pdf = PdfReader(original_pdf_path)
overlay_pdf = PdfReader(packet)

for i, page in enumerate(base_pdf.pages):
    merger = PageMerge(page)
    merger.add(overlay_pdf.pages[0]).render()

PdfWriter(output_pdf_path, trailer=base_pdf).write()
print(f"✅ Filled form saved as: {output_pdf_path}")


✅ Filled form saved as: /kaggle/working/final_filled_form.pdf
